In [1]:
!pip3 install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu
!pip install accelerate peft bitsandbytes transformers langchain chromadb --quiet
!pip install pypdf --quiet
!pip install tiktoken --quiet
!pip install sentence_transformers lark auto-gptq optimum --quiet
!pip install cohere
!pip install openai
!pip install sentence_transformers
!pip install transformers
!pip install --upgrade accelerate
!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

Looking in indexes: https://download.pytorch.org/whl/nightly/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.3/230.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ls

drive  sample_data


In [4]:
file_path='drive/MyDrive/test-pipeline/pak-suzuki-car-reviews.csv'

In [5]:
import pandas as pd
df = pd.read_csv(file_path)
df.head()

,Unnamed: 0,Model,Overall_rating,Date,Review,Style,Comfort,Fuel_Economy,Performance,Value_for_Money
0,0,2023 Suzuki Bolan,1,"Nov 30, 2023",\nis gadi k baray me 3 lafaz kahunga 1 bakwaaa...,1,1,1,1,1
1,1,2022 Suzuki Swift GLX CVT,4,"Nov 29, 2023","\nA good looking car with best fuel ? economy,...",5,4,5,4,5
2,2,2022 Suzuki Cultus Auto Gear Shift,5,"Nov 26, 2023",\nfor me i would must buy because its will sav...,5,5,5,5,5
3,3,2007 Suzuki Liana RXi (CNG),5,"Nov 12, 2023",\nI have used it for more than 5 years. litera...,5,5,5,5,5
4,4,2022 Suzuki Alto VX,4,"Nov 09, 2023",\nExterior: 10/10 on design but the quality us...,5,5,5,4,5


In [6]:
df = df.drop(['Unnamed: 0'],axis = 1)
df.head()

,Model,Overall_rating,Date,Review,Style,Comfort,Fuel_Economy,Performance,Value_for_Money
0,2023 Suzuki Bolan,1,"Nov 30, 2023",\nis gadi k baray me 3 lafaz kahunga 1 bakwaaa...,1,1,1,1,1
1,2022 Suzuki Swift GLX CVT,4,"Nov 29, 2023","\nA good looking car with best fuel ? economy,...",5,4,5,4,5
2,2022 Suzuki Cultus Auto Gear Shift,5,"Nov 26, 2023",\nfor me i would must buy because its will sav...,5,5,5,5,5
3,2007 Suzuki Liana RXi (CNG),5,"Nov 12, 2023",\nI have used it for more than 5 years. litera...,5,5,5,5,5
4,2022 Suzuki Alto VX,4,"Nov 09, 2023",\nExterior: 10/10 on design but the quality us...,5,5,5,4,5


In [7]:
import re

def preprocess_review(text):
  # Remove "More" and "Less" at the end of the text
    cleaned_text = re.sub(r'\s*More\s*Less\s*', '', text)

    # Remove extra white spaces
    cleaned_text = ' '.join(cleaned_text.split()).replace('\n', '').replace('\t', '')

    return cleaned_text

In [8]:
preprocessed_df = df.copy()
preprocessed_df['Review'] = preprocessed_df['Review'].apply(preprocess_review)

In [9]:
preprocessed_df.head()

,Model,Overall_rating,Date,Review,Style,Comfort,Fuel_Economy,Performance,Value_for_Money
0,2023 Suzuki Bolan,1,"Nov 30, 2023",is gadi k baray me 3 lafaz kahunga 1 bakwaaas ...,1,1,1,1,1
1,2022 Suzuki Swift GLX CVT,4,"Nov 29, 2023","A good looking car with best fuel ? economy,Dr...",5,4,5,4,5
2,2022 Suzuki Cultus Auto Gear Shift,5,"Nov 26, 2023",for me i would must buy because its will save ...,5,5,5,5,5
3,2007 Suzuki Liana RXi (CNG),5,"Nov 12, 2023",I have used it for more than 5 years. literall...,5,5,5,5,5
4,2022 Suzuki Alto VX,4,"Nov 09, 2023",Exterior: 10/10 on design but the quality used...,5,5,5,4,5


In [10]:
pre_processed_file_path = 'drive/MyDrive/test-pipeline/pre-processed-pak-suzuki-car-reviews.csv'
# saving the dataframe
preprocessed_df.to_csv(pre_processed_file_path)

In [11]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(pre_processed_file_path)
documents = loader.load()

In [12]:

from huggingface_hub import notebook_login
notebook_login()

In [13]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

# encoder
model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.2k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [14]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(documents, embedding=embeddings, persist_directory='./llama-db')

In [15]:
import torch
import transformers
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain
from transformers import AutoTokenizer, AutoModelForCausalLM , AutoModel

In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/Llama-2-7b-Chat-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

prompt = "What is a car?"
prompt_template=f'''[INST] <<SYS>>
You are a helpful, respectful and honest assistant. You will be dealing with questions related to Automobiles domain. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
{prompt}[/INST]

'''

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt_template)[0]['generated_text'])
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})


model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

*** Pipeline:


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


[INST] <<SYS>>
You are a helpful, respectful and honest assistant. You will be dealing with questions related to Automobiles domain. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
What is a car?[/INST]

Thank you for asking! A car, also known as an automobile, is a vehicle designed for transporting people and goods on roads. It typically has four wheels and an engine that powers its movement. Cars come in various shapes, sizes, and styles, ranging from small hatchbacks to large SUVs and luxury vehicles. They provide a convenient mode of transportation for individuals and fami

In [ ]:
# will be executed when actually made a chat app

from langchain import PromptTemplate

prompt_template_2="""
You are a helpful, respectful and honest assistant. You will be dealing with questions related to Automobiles domain. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
And in the end also encourage the user to ask another question if he or she has.

Context: {context}
Question: {prompt}
"""

# prompt = PromptTemplate.from_template(template=prompt_template_2)
# prompt_formated_str = prompt.format(prompt="How does the perception of 'Value_for_Money' vary across different models?")

custom_prompt_temp = PromptTemplate(input_variables=["prompt","context"], template=prompt_template_2)

In [18]:
# prompt: create rag and pass the QA_CHAIN_PROMPT template

from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    # chain_type_kwargs={"prompt": custom_prompt_temp},
    retriever=db.as_retriever(),
    return_source_documents=True,
)

# Use the chain to answer a question
# question = "How does the perception of 'Value_for_Money' vary across different models?"

# response = qa_chain({"prompt": question})
# print(response)


In [19]:
print(qa_chain("How does the perception of 'Value_for_Money' vary across different models?")['result'])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 Thank you for asking! Based on the reviews provided, here's how the perception of 'Value_for_Money' varies across different models:
* Model: 2011 Suzuki Kizashi Base Grade - Perception of Value_for_Money is 2.
* Model: 1991 Suzuki Margalla - Perception of Value_for_Money is 5.
* Model: 2013 Suzuki Mehran VXR Euro II - Perception of Value_for_Money is 3.
* Model: 873 - Perception of Value_for_Money is 5.
It's worth noting that these ratings are based on the opinions of individual reviewers and may not reflect the overall perception of the brand or model. Additionally, these ratings may also vary based on factors like location, personal preferences, and market conditions.


In [20]:
print(qa_chain("Which car models receive the highest ratings in different categories?")['result'])

 The following car models received the highest ratings in different categories based on the reviews provided:
* Best Overall Rating: 2007 Suzuki Liana RXi (CNG) with an overall rating of 527
* Best Style Rating: 2007 Suzuki Liana RXi (CNG) with a style rating of 5
* Best Comfort Rating: 2007 Suzuki Liana RXi (CNG) with a comfort rating of 5
* Best Fuel Economy Rating: 2007 Suzuki Liana RXi (CNG) with a fuel economy rating of 5.
* Best Performance Rating: 2007 Suzuki Liana RXi (CNG) with a performance rating of 5.
* Best Value For Money Rating: 2007 Suzuki Liana RXi (CNG) with a value for money rating of 5.


In [21]:
print(qa_chain("What are the temporal trends in user reviews? Are there any noticeable patterns or shifts over time?")['result'])


There are some noticeable patterns and shifts in the user reviews over time. Here are some observations:
1. Improving ratings: Overall ratings of Suzuki Cultus have improved over time. In 2017, the overall rating was 4, in 2019, it became 4.5, and in 2020, it reached 4.8. Similarly, the ratings of Suzuki Alto and Swift have also improved.
2. Shift towards new models: The reviews of newer models, such as Cultus 2017 and Swift 2014, are generally more positive than older models. For example, the review of Cultus 2017 is 4.8 out of 5, while the review of Cultus 2009 is 3.8 out of 5.
3. Decrease in fuel economy complaints: There is a slight decrease in the number of users reporting fuel economy issues over time. In 2009, 15% of users reported fuel economy issues, while in 2017 and 2019, only 5% of users reported them.
4. More focus on comfort: As time goes by, users are more focused on comfort features in their reviews. In 2009, only 3% of users mentioned comfort, while in 2017 and 2019, 

In [22]:
print(qa_chain("How does the 'Overall_rating' correlate with individual aspects like Style, Comfort, Fuel Economy, Performance, and Value for Money?")['result'])

 The overall rating given by the reviewer is based on their combined assessment of various aspects mentioned above, including style, comfort, fuel economy, performance, and value for money. Therefore, a higher overall rating generally indicates a positive overall experience with the vehicle.
